In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter

def rgb_to_hsv_arr(img):
    """
    Convert RGB image (0-1) to HSV (H: 0-360, S: 0-1, V: 0-1)
    """
    r, g, b = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    cmax = np.maximum.reduce([r, g, b])
    cmin = np.minimum.reduce([r, g, b])
    delta = cmax - cmin

    h = np.zeros_like(cmax)
    mask_r = (cmax == r) & (delta > 0)
    h[mask_r] = 60 * (((g - b) / delta)[mask_r] % 6)
    mask_g = (cmax == g) & (delta > 0)
    h[mask_g] = 60 * (((b - r) / delta)[mask_g] + 2)
    mask_b = (cmax == b) & (delta > 0)
    h[mask_b] = 60 * (((r - g) / delta)[mask_b] + 4)

    s = np.zeros_like(cmax)
    mask_s = cmax > 0
    s[mask_s] = delta[mask_s] / cmax[mask_s]

    v = cmax

    return np.stack([h, s, v], axis=-1)

def hsv_to_rgb_arr(hsv):
    """
    Convert HSV (H: 0-360, S: 0-1, V: 0-1) back to RGB (0-1)
    """
    h, s, v = hsv[:, :, 0], hsv[:, :, 1], hsv[:, :, 2]
    h = h % 360
    c = v * s
    x = c * (1 - np.abs((h / 60) % 2 - 1))
    m = v - c

    r = np.zeros_like(h)
    g = np.zeros_like(h)
    b = np.zeros_like(h)

    mask1 = (0 <= h) & (h < 60)
    r[mask1], g[mask1], b[mask1] = c[mask1], x[mask1], 0

    mask2 = (60 <= h) & (h < 120)
    r[mask2], g[mask2], b[mask2] = x[mask2], c[mask2], 0

    mask3 = (120 <= h) & (h < 180)
    r[mask3], g[mask3], b[mask3] = 0, c[mask3], x[mask3]

    mask4 = (180 <= h) & (h < 240)
    r[mask4], g[mask4], b[mask4] = 0, x[mask4], c[mask4]

    mask5 = (240 <= h) & (h < 300)
    r[mask5], g[mask5], b[mask5] = x[mask5], 0, c[mask5]

    mask6 = (300 <= h) & (h < 360)
    r[mask6], g[mask6], b[mask6] = c[mask6], 0, x[mask6]

    rgb = np.stack([r, g, b], axis=-1) + m[:, :, np.newaxis]
    return np.clip(rgb, 0, 1)

def rgb_to_ycbcr(img):
    """
    Convert RGB (0-1) to YCbCr (0-1)
    """
    r, g, b = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    y = 0.299 * r + 0.587 * g + 0.114 * b
    cb = -0.168736 * r - 0.331264 * g + 0.5 * b + 0.5
    cr = 0.5 * r - 0.418688 * g - 0.081312 * b + 0.5
    return np.stack([y, cb, cr], axis=-1)

def enhance_skin_tone(input_path, output_path):
    """
    Enhances the skin tone in the image by detecting skin regions, smoothing them,
    and adjusting brightness and saturation for a more beautiful appearance.
    Focuses only on skin areas.
    """
    # Load the image (assumes RGB, no alpha)
    img = plt.imread(input_path)
    if img.ndim != 3 or img.shape[2] != 3:
        raise ValueError("Image must be RGB (no alpha channel).")

    # Ensure float 0-1
    if img.dtype == np.uint8:
        img = img / 255.0

    # Detect skin using YCbCr color space (better for skin detection across tones)
    ycbcr = rgb_to_ycbcr(img)
    y, cb, cr = ycbcr[:, :, 0] * 255, ycbcr[:, :, 1] * 255, ycbcr[:, :, 2] * 255

    # Broad range to capture various skin tones (adjust if needed for specific cases)
    skin_mask = (y > 50) & (cb > 75) & (cb < 130) & (cr > 130) & (cr < 170)

    # Smooth the mask to avoid harsh edges
    skin_mask_smooth = gaussian_filter(skin_mask.astype(float), sigma=3)
    skin_mask = skin_mask_smooth > 0.3  # Lower threshold for broader inclusion

    # Smooth the skin areas (bilateral-like using Gaussian on masked image)
    smoothed = np.copy(img)
    for channel in range(3):
        masked_img = img[:, :, channel] * skin_mask_smooth
        smoothed_channel = gaussian_filter(masked_img, sigma=2) / (gaussian_filter(skin_mask_smooth, sigma=2) + 1e-6)
        smoothed[:, :, channel] = smoothed_channel + img[:, :, channel] * (1 - skin_mask_smooth)

    # Convert smoothed image to HSV for tone enhancement
    hsv = rgb_to_hsv_arr(smoothed)

    # Enhance skin tone: increase brightness (V) and saturation (S) slightly for vibrancy
    # Apply only to skin areas
    hsv[:, :, 1][skin_mask] = np.clip(hsv[:, :, 1][skin_mask] * 1.15, 0, 1)  # Boost saturation
    hsv[:, :, 2][skin_mask] = np.clip(hsv[:, :, 2][skin_mask] * 1.10, 0, 1)  # Boost brightness

    # Convert back to RGB
    enhanced_rgb = hsv_to_rgb_arr(hsv)

    # Save the enhanced image
    plt.imsave(output_path, enhanced_rgb)

# Example usage
if __name__ == "__main__":
    # Replace with your file paths
    input_image = r"C:/Users/afolabi.faruq/Documents/colsmile/picture.jpg"  # Path to your uploaded/input image
    output_image = "enhanced.jpg"  # Path for the output
    enhance_skin_tone(input_image, output_image)
    print(f"Enhanced image saved to {output_image}")

FileNotFoundError: [Errno 2] No such file or directory: 'picture.jpg'

In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import gaussian_filter

def rgb_to_hsv_arr(img):
    """
    Convert RGB image (0-1) to HSV (H: 0-360, S: 0-1, V: 0-1)
    """
    r, g, b = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    cmax = np.maximum.reduce([r, g, b])
    cmin = np.minimum.reduce([r, g, b])
    delta = cmax - cmin

    h = np.zeros_like(cmax)
    mask_r = (cmax == r) & (delta > 0)
    h[mask_r] = 60 * (((g - b) / delta)[mask_r] % 6)
    mask_g = (cmax == g) & (delta > 0)
    h[mask_g] = 60 * (((b - r) / delta)[mask_g] + 2)
    mask_b = (cmax == b) & (delta > 0)
    h[mask_b] = 60 * (((r - g) / delta)[mask_b] + 4)

    s = np.zeros_like(cmax)
    mask_s = cmax > 0
    s[mask_s] = delta[mask_s] / cmax[mask_s]

    v = cmax

    return np.stack([h, s, v], axis=-1)

def hsv_to_rgb_arr(hsv):
    """
    Convert HSV (H: 0-360, S: 0-1, V: 0-1) back to RGB (0-1)
    """
    h, s, v = hsv[:, :, 0], hsv[:, :, 1], hsv[:, :, 2]
    h = h % 360
    c = v * s
    x = c * (1 - np.abs((h / 60) % 2 - 1))
    m = v - c

    r = np.zeros_like(h)
    g = np.zeros_like(h)
    b = np.zeros_like(h)

    mask1 = (0 <= h) & (h < 60)
    r[mask1], g[mask1], b[mask1] = c[mask1], x[mask1], 0

    mask2 = (60 <= h) & (h < 120)
    r[mask2], g[mask2], b[mask2] = x[mask2], c[mask2], 0

    mask3 = (120 <= h) & (h < 180)
    r[mask3], g[mask3], b[mask3] = 0, c[mask3], x[mask3]

    mask4 = (180 <= h) & (h < 240)
    r[mask4], g[mask4], b[mask4] = 0, x[mask4], c[mask4]

    mask5 = (240 <= h) & (h < 300)
    r[mask5], g[mask5], b[mask5] = x[mask5], 0, c[mask5]

    mask6 = (300 <= h) & (h < 360)
    r[mask6], g[mask6], b[mask6] = c[mask6], 0, x[mask6]

    rgb = np.stack([r, g, b], axis=-1) + m[:, :, np.newaxis]
    return np.clip(rgb, 0, 1)

def rgb_to_ycbcr(img):
    """
    Convert RGB (0-1) to YCbCr (0-1)
    """
    r, g, b = img[:, :, 0], img[:, :, 1], img[:, :, 2]
    y = 0.299 * r + 0.587 * g + 0.114 * b
    cb = -0.168736 * r - 0.331264 * g + 0.5 * b + 0.5
    cr = 0.5 * r - 0.418688 * g - 0.081312 * b + 0.5
    return np.stack([y, cb, cr], axis=-1)

def enhance_skin_tone(input_path, output_path):
    """
    Enhances the skin tone in the image by detecting skin regions, smoothing them,
    and adjusting brightness and saturation for a more beautiful appearance.
    Focuses only on skin areas.
    """
    # Load the image (assumes RGB, no alpha)
    img = plt.imread(input_path)
    if img.ndim != 3 or img.shape[2] != 3:
        raise ValueError("Image must be RGB (no alpha channel).")

    # Ensure float 0-1
    if img.dtype == np.uint8:
        img = img / 255.0

    # Detect skin using YCbCr color space (better for skin detection across tones)
    ycbcr = rgb_to_ycbcr(img)
    y, cb, cr = ycbcr[:, :, 0] * 255, ycbcr[:, :, 1] * 255, ycbcr[:, :, 2] * 255

    # Broad range to capture various skin tones (adjust if needed for specific cases)
    skin_mask = (y > 50) & (cb > 75) & (cb < 130) & (cr > 130) & (cr < 170)

    # Smooth the mask to avoid harsh edges
    skin_mask_smooth = gaussian_filter(skin_mask.astype(float), sigma=3)
    skin_mask = skin_mask_smooth > 0.3  # Lower threshold for broader inclusion

    # Smooth the skin areas (bilateral-like using Gaussian on masked image)
    smoothed = np.copy(img)
    for channel in range(3):
        masked_img = img[:, :, channel] * skin_mask_smooth
        smoothed_channel = gaussian_filter(masked_img, sigma=2) / (gaussian_filter(skin_mask_smooth, sigma=2) + 1e-6)
        smoothed[:, :, channel] = smoothed_channel + img[:, :, channel] * (1 - skin_mask_smooth)

    # Convert smoothed image to HSV for tone enhancement
    hsv = rgb_to_hsv_arr(smoothed)

    # Enhance skin tone: increase brightness (V) and saturation (S) slightly for vibrancy
    # Apply only to skin areas
    hsv[:, :, 1][skin_mask] = np.clip(hsv[:, :, 1][skin_mask] * 1.15, 0, 1)  # Boost saturation
    hsv[:, :, 2][skin_mask] = np.clip(hsv[:, :, 2][skin_mask] * 1.10, 0, 1)  # Boost brightness

    # Convert back to RGB
    enhanced_rgb = hsv_to_rgb_arr(hsv)

    # Save the enhanced image
    plt.imsave(output_path, enhanced_rgb)

# Example usage
if __name__ == "__main__":
    # Replace with your file paths
    input_image = "./picture.jpg"  # Path to your uploaded/input image
    output_image = "enhanced.jpg"  # Path for the output
    enhance_skin_tone(input_image, output_image)
    print(f"Enhanced image saved to {output_image}")

FileNotFoundError: [Errno 2] No such file or directory: './picture.jpg'

In [ ]:
import cv2
import numpy as np
from PIL import Image

def snapchat_enhance(image_path, output_path="enhanced_output.jpg"):
    # Load image
    img = cv2.imread(image_path)
    if img is None:
        raise ValueError("Image not found or invalid path")

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # ----- Step 1: Gentle face/skin smoothing -----
    smooth = cv2.bilateralFilter(img, d=15, sigmaColor=75, sigmaSpace=75)

    # ----- Step 2: Create detail layer (for clarity pop) -----
    gaussian = cv2.GaussianBlur(smooth, (0, 0), sigmaX=10)
    detail = cv2.addWeighted(smooth, 1.5, gaussian, -0.5, 0)

    # ----- Step 3: Add warmth -----
    warm = detail.copy().astype(np.float32)
    warm[:, :, 0] *= 0.95
    warm[:, :, 1] *= 1.05
    warm[:, :, 2] *= 1.10
    warm = np.clip(warm, 0, 255).astype(np.uint8)

    # ----- Step 4: Light HDR effect -----
    hdr = cv2.detailEnhance(warm, sigma_s=12, sigma_r=0.15)

    # ----- Step 5: Light sharpening -----
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharp = cv2.filter2D(hdr, -1, kernel)

    # Convert back to BGR for saving
    result = cv2.cvtColor(sharp, cv2.COLOR_RGB2BGR)
    cv2.imwrite(output_path, result)

    print(f"Enhanced image saved as {output_path}")

    return output_path

# --------------------------
# Example usage:
# --------------------------
snapchat_enhance("picture.jpg", "output.jpg")


Enhanced image saved as output.jpg


'output.jpg'